# 📘 Day 1: Reinforcement Learning Fundamentals

**🎯 Goal:** Master RL fundamentals - how AI agents learn by trial and error (like humans!)

**⏱️ Time:** 90-120 minutes

**🌟 Why This Matters for AI:**
- RL powers self-driving cars, robotics, and game-playing AI
- ChatGPT uses RLHF (Reinforcement Learning from Human Feedback) for alignment
- AlphaGo beat world champion using RL (40 million self-play games!)
- Autonomous agents and multi-agent systems use RL
- Real-world optimization: energy grids, traffic systems, resource allocation
- Foundation of AGI research - learning from interaction
- Used in robotics (Boston Dynamics), games (OpenAI Five), and drug discovery

---

## 🤔 What is Reinforcement Learning?

**Reinforcement Learning = Learning by trial and error through rewards and punishments**

**Human Analogy:**
Learning to ride a bike:
- **Try action:** Pedal and steer
- **Get feedback:** Stay balanced (+reward) OR fall (-punishment)
- **Learn:** Adjust actions to maximize staying upright
- **Improve:** Eventually ride without falling!

**In AI:**
- **Agent:** The learner/decision maker (you on the bike)
- **Environment:** The world the agent interacts with (road, gravity)
- **Actions:** Choices the agent makes (pedal, steer left/right)
- **State:** Current situation (speed, balance, position)
- **Reward:** Feedback signal (+1 for staying balanced, -1 for falling)
- **Policy:** Strategy for choosing actions (what to do in each state)

### 🎯 RL vs Supervised Learning vs Unsupervised Learning

| Feature | Supervised Learning | Unsupervised Learning | Reinforcement Learning |
|---------|---------------------|----------------------|------------------------|
| **Learning from** | Labeled examples | Unlabeled data | Trial and error |
| **Feedback** | Correct answer given | No feedback | Reward/punishment |
| **Goal** | Predict labels | Find patterns | Maximize cumulative reward |
| **Example** | Image classification | Clustering | Game playing |
| **Training data** | (X, Y) pairs | X only | (State, Action, Reward) sequences |

### 🎯 Real-World Applications (2024-2025)

**Where RL is used:**
1. **ChatGPT/Claude:** RLHF fine-tunes models to be helpful and safe
2. **Self-Driving Cars:** Learn to navigate traffic (Tesla, Waymo)
3. **Robotics:** Boston Dynamics robots learn to walk, jump, dance
4. **Game AI:** AlphaGo, OpenAI Five (Dota 2), DeepMind's AlphaStar (StarCraft)
5. **Recommendation Systems:** YouTube, Netflix optimize for engagement
6. **Resource Optimization:** Google data centers (40% energy savings!)
7. **Trading Algorithms:** Financial markets
8. **Healthcare:** Treatment optimization, drug discovery

**The Revolution:**
- **2013:** DeepMind's DQN plays Atari games at human level
- **2016:** AlphaGo beats Lee Sedol (Go world champion)
- **2017:** AlphaZero masters Chess, Shogi, Go from scratch
- **2019:** OpenAI Five beats Dota 2 world champions
- **2022:** ChatGPT uses RLHF for alignment
- **2024-2025:** RL powers autonomous agents and robotics breakthroughs

Let's build an RL agent from scratch! 👇

In [ ]:
# Import essential libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque
import random
from IPython.display import clear_output
import time

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Make plots beautiful
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("Let's build RL agents from scratch! 🚀")

## 🧩 The RL Framework: Agent-Environment Interaction

**The RL Loop:**

```
     ┌──────────────┐
     │    Agent     │
     │  (Learner)   │
     └──────┬───────┘
            │
       Action (a_t)
            │
            ↓
     ┌──────────────┐
     │ Environment  │
     │   (World)    │
     └──────┬───────┘
            │
    State (s_t+1), Reward (r_t+1)
            │
            ↓
     [Repeat forever]
```

**At each time step t:**
1. **Agent** observes state s_t
2. **Agent** takes action a_t (based on policy π)
3. **Environment** transitions to new state s_t+1
4. **Environment** gives reward r_t+1
5. **Agent** learns from experience (s_t, a_t, r_t+1, s_t+1)
6. **Repeat!**

### 🎯 Key Concepts:

**1. State (s):**
- Complete description of the environment
- Example (Grid world): (row, col) position
- Example (Chess): Board configuration
- Example (Self-driving): Speed, position, nearby cars, traffic lights

**2. Action (a):**
- Choice the agent makes
- Example (Grid world): {UP, DOWN, LEFT, RIGHT}
- Example (Chess): Move a piece
- Example (Self-driving): {Accelerate, Brake, Turn}

**3. Reward (r):**
- Immediate feedback signal
- Example (Grid world): +10 for goal, -1 for each step
- Example (Chess): +1 for win, 0 for draw, -1 for loss
- Example (Self-driving): +1 for safe driving, -100 for crash

**4. Policy (π):**
- Strategy for choosing actions
- π(s) = action to take in state s
- Goal: Find optimal policy π* that maximizes rewards!

**5. Value Function (V):**
- Expected total reward from a state
- V(s) = "How good is this state?"
- Considers future rewards, not just immediate

Let's implement a simple environment!

In [ ]:
# Simple Grid World Environment

class GridWorld:
    """
    Simple 4x4 grid world:
    - Agent starts at (0, 0)
    - Goal at (3, 3)
    - Walls at (1, 1) and (2, 2)
    - Actions: UP, DOWN, LEFT, RIGHT
    - Rewards: +10 for goal, -1 for each step, -5 for hitting wall
    """
    
    def __init__(self, size=4):
        self.size = size
        self.start = (0, 0)
        self.goal = (size-1, size-1)
        self.walls = [(1, 1), (2, 2)]  # Obstacles
        self.actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        self.action_effects = {
            'UP': (-1, 0),
            'DOWN': (1, 0),
            'LEFT': (0, -1),
            'RIGHT': (0, 1)
        }
        self.reset()
    
    def reset(self):
        """Reset to starting position"""
        self.state = self.start
        return self.state
    
    def step(self, action):
        """
        Take action and return (next_state, reward, done)
        """
        # Calculate next position
        delta = self.action_effects[action]
        next_state = (self.state[0] + delta[0], self.state[1] + delta[1])
        
        # Check if next state is valid
        if self._is_valid(next_state):
            self.state = next_state
            reward = -1  # Step cost
        else:
            # Hit wall or boundary - stay in place
            reward = -5  # Penalty for hitting wall
        
        # Check if reached goal
        done = False
        if self.state == self.goal:
            reward = 10  # Goal reward!
            done = True
        
        return self.state, reward, done
    
    def _is_valid(self, state):
        """Check if state is within bounds and not a wall"""
        row, col = state
        
        # Check bounds
        if row < 0 or row >= self.size or col < 0 or col >= self.size:
            return False
        
        # Check walls
        if state in self.walls:
            return False
        
        return True
    
    def render(self):
        """Visualize the grid world"""
        grid = np.zeros((self.size, self.size))
        
        # Mark walls
        for wall in self.walls:
            grid[wall] = -1
        
        # Mark goal
        grid[self.goal] = 2
        
        # Mark agent
        grid[self.state] = 1
        
        # Plot
        plt.figure(figsize=(6, 6))
        plt.imshow(grid, cmap='RdYlGn', vmin=-1, vmax=2)
        
        # Add grid lines
        for i in range(self.size + 1):
            plt.axhline(i - 0.5, color='black', linewidth=2)
            plt.axvline(i - 0.5, color='black', linewidth=2)
        
        # Add labels
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.state:
                    plt.text(j, i, '🤖', ha='center', va='center', fontsize=30)
                elif (i, j) == self.goal:
                    plt.text(j, i, '🎯', ha='center', va='center', fontsize=30)
                elif (i, j) in self.walls:
                    plt.text(j, i, '🧱', ha='center', va='center', fontsize=30)
        
        plt.xlim(-0.5, self.size - 0.5)
        plt.ylim(self.size - 0.5, -0.5)
        plt.xticks([])
        plt.yticks([])
        plt.title('Grid World: 🤖=Agent, 🎯=Goal, 🧱=Wall', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

# Create environment
env = GridWorld(size=4)

print("✅ Grid World Environment Created!")
print(f"\nEnvironment Details:")
print(f"  Grid size: {env.size}x{env.size}")
print(f"  Start: {env.start}")
print(f"  Goal: {env.goal}")
print(f"  Walls: {env.walls}")
print(f"  Actions: {env.actions}")
print(f"\nRewards:")
print(f"  +10 for reaching goal")
print(f"  -1 for each step (encourages shortest path)")
print(f"  -5 for hitting wall/boundary")

# Visualize initial state
env.render()

In [ ]:
# Test the environment with random actions

print("🎮 Testing environment with random actions...\n")

state = env.reset()
print(f"Initial state: {state}")

total_reward = 0
for step in range(10):
    # Random action
    action = random.choice(env.actions)
    
    # Take step
    next_state, reward, done = env.step(action)
    total_reward += reward
    
    print(f"Step {step + 1}: Action={action:6s}, State={next_state}, Reward={reward:3}, Done={done}")
    
    if done:
        print(f"\n🎉 Reached goal in {step + 1} steps!")
        break

print(f"\nTotal reward: {total_reward}")
print("\n💡 Notice: Random actions are inefficient! We need a smarter policy.")

## 🎲 Markov Decision Processes (MDPs)

**MDP = Mathematical framework for modeling RL problems**

### Definition:

An MDP is defined by a tuple (S, A, P, R, γ):

**1. S:** Set of states
- All possible situations
- Example: All (row, col) positions in grid

**2. A:** Set of actions
- All possible choices
- Example: {UP, DOWN, LEFT, RIGHT}

**3. P:** Transition probability P(s'|s,a)
- Probability of reaching state s' from state s after action a
- Example: In deterministic grid, P = 1.0 for intended direction
- Example: In stochastic grid, P = 0.8 for intended, 0.1 for each side

**4. R:** Reward function R(s,a,s')
- Immediate reward for transition
- Example: +10 for goal, -1 for step

**5. γ (gamma):** Discount factor (0 ≤ γ ≤ 1)
- How much we value future rewards
- γ = 0: Only care about immediate reward (myopic)
- γ = 1: Future rewards as important as immediate (far-sighted)
- γ = 0.9: Common choice (balance)

### 🎯 The Markov Property:

**"The future is independent of the past given the present"**

Mathematically:
```
P(s_t+1 | s_t, s_t-1, s_t-2, ..., s_0) = P(s_t+1 | s_t)
```

**Meaning:**
- Current state contains all relevant information
- Don't need to remember entire history
- Simplifies learning significantly!

**Example:**
- ✅ Chess: Current board position is enough (Markov)
- ❌ Poker: Need to remember previous bets/actions (Partially Observable)

### 🎯 Goal: Maximize Return

**Return (G_t):** Total discounted reward from time t

```
G_t = r_t+1 + γ*r_t+2 + γ²*r_t+3 + γ³*r_t+4 + ...
    = Σ γ^k * r_t+k+1  (sum from k=0 to ∞)
```

**Example (γ=0.9):**
```
Rewards: [1, 1, 1, 10]
Return = 1 + 0.9*1 + 0.81*1 + 0.729*10 = 10.0
```

**Why discount?**
1. **Uncertainty:** Future is uncertain
2. **Mathematical convenience:** Ensures finite return
3. **Preference:** Immediate rewards preferred (human psychology!)
4. **Convergence:** Helps algorithms converge

Let's visualize MDPs!

In [ ]:
# Visualize discount factor effect

def calculate_return(rewards, gamma):
    """Calculate discounted return"""
    G = 0
    for t, r in enumerate(rewards):
        G += (gamma ** t) * r
    return G

# Example reward sequences
rewards_immediate = [10, 0, 0, 0, 0]  # Immediate reward
rewards_delayed = [0, 0, 0, 0, 10]    # Delayed reward

gammas = np.linspace(0, 1, 20)
returns_immediate = [calculate_return(rewards_immediate, g) for g in gammas]
returns_delayed = [calculate_return(rewards_delayed, g) for g in gammas]

plt.figure(figsize=(12, 5))

# Plot 1: Return vs Gamma
plt.subplot(1, 2, 1)
plt.plot(gammas, returns_immediate, 'b-o', label='Immediate reward [10,0,0,0,0]', linewidth=2)
plt.plot(gammas, returns_delayed, 'r-s', label='Delayed reward [0,0,0,0,10]', linewidth=2)
plt.xlabel('Discount Factor (γ)', fontsize=12)
plt.ylabel('Total Return', fontsize=12)
plt.title('Effect of Discount Factor on Return', fontsize=13, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.axvline(0.9, color='green', linestyle='--', alpha=0.5, label='Common choice')

# Plot 2: Discount weights over time
plt.subplot(1, 2, 2)
time_steps = np.arange(20)
for gamma in [0.5, 0.9, 0.99]:
    weights = [gamma ** t for t in time_steps]
    plt.plot(time_steps, weights, 'o-', label=f'γ={gamma}', linewidth=2)

plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Discount Weight (γ^t)', fontsize=12)
plt.title('How Much Future Rewards Count', fontsize=13, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Observations:")
print("  Left plot: Higher γ → delayed rewards matter more")
print("  Right plot: Lower γ → only nearby rewards matter")
print("\n💡 Common choices:")
print("  γ = 0.9:  Games, robotics (balance short/long term)")
print("  γ = 0.99: Finance, long-term planning")
print("  γ = 0.5:  Very myopic (rare in practice)")

## 🧠 Q-Learning Algorithm

**Q-Learning = Learn action-values (Q-values) to find optimal policy**

### What is Q?

**Q(s, a) = Expected return from taking action a in state s, then following optimal policy**

- Q stands for "Quality" of action
- Q(s, a) tells us: "How good is action a in state s?"
- Higher Q → better action!

**Example (Grid World):**
```
State (1, 1), next to goal (1, 2):
  Q(s, RIGHT) = 9  (leads to goal!)
  Q(s, LEFT)  = -5 (moves away)
  Q(s, UP)    = -3
  Q(s, DOWN)  = -4
  
Best action: RIGHT (highest Q-value)
```

### The Bellman Equation:

**Optimal Q-value satisfies:**
```
Q*(s, a) = R(s,a) + γ * max_a' Q*(s', a')
```

**In words:**
- Q-value = Immediate reward + Discounted best future Q-value
- Recursive definition (dynamic programming!)

### Q-Learning Update Rule:

**After taking action a in state s, receiving reward r, reaching state s':**

```
Q(s, a) ← Q(s, a) + α * [r + γ * max_a' Q(s', a') - Q(s, a)]
                        ︸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━︸
                                  TD Error (δ)
```

**Components:**
- **α (alpha):** Learning rate (0 < α ≤ 1)
  - α = 0.1: Slow, stable learning
  - α = 0.5: Faster learning
  - α = 1.0: Only remember latest sample

- **TD Error (δ):** How much we were wrong
  - δ > 0: We underestimated → increase Q
  - δ < 0: We overestimated → decrease Q

- **Target:** r + γ * max_a' Q(s', a')
  - What Q(s,a) "should be" based on new experience

### 🎯 Q-Learning Algorithm:

```python
1. Initialize Q(s, a) = 0 for all state-action pairs
2. For each episode:
     a. Reset environment to start state s
     b. While not done:
          i.   Choose action a using ε-greedy policy
          ii.  Take action a, observe r, s'
          iii. Update: Q(s,a) ← Q(s,a) + α[r + γ*max_a' Q(s',a') - Q(s,a)]
          iv.  s ← s'
3. Return learned Q-table
```

### 🎯 Exploration vs Exploitation:

**The Dilemma:**
- **Exploit:** Choose action with highest Q-value (greedy)
- **Explore:** Try random actions (discover better strategies)

**ε-greedy Policy:**
- With probability ε: Random action (explore)
- With probability 1-ε: Best action (exploit)

**Example:**
```
ε = 0.1:
  - 10% of time: explore (random)
  - 90% of time: exploit (greedy)
```

**Decay ε over time:**
- Start high (ε = 1.0): Explore a lot
- End low (ε = 0.01): Mostly exploit
- Balances exploration and exploitation!

Let's implement Q-Learning!

In [ ]:
# Q-Learning Agent

class QLearningAgent:
    """
    Q-Learning agent for grid world
    """
    
    def __init__(self, actions, learning_rate=0.1, discount_factor=0.9, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
        self.actions = actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Q-table: dictionary {state: {action: Q-value}}
        self.q_table = defaultdict(lambda: {action: 0.0 for action in actions})
        
    def get_action(self, state, training=True):
        """
        Choose action using ε-greedy policy
        """
        if training and random.random() < self.epsilon:
            # Explore: random action
            return random.choice(self.actions)
        else:
            # Exploit: best action
            q_values = self.q_table[state]
            max_q = max(q_values.values())
            # Handle ties: randomly choose among best actions
            best_actions = [a for a in self.actions if q_values[a] == max_q]
            return random.choice(best_actions)
    
    def update(self, state, action, reward, next_state, done):
        """
        Update Q-value using Q-learning rule
        """
        # Current Q-value
        current_q = self.q_table[state][action]
        
        # Calculate target
        if done:
            target = reward  # No future rewards
        else:
            # Best Q-value for next state
            max_next_q = max(self.q_table[next_state].values())
            target = reward + self.gamma * max_next_q
        
        # TD error
        td_error = target - current_q
        
        # Update Q-value
        self.q_table[state][action] = current_q + self.lr * td_error
        
        return td_error
    
    def decay_epsilon(self):
        """Decay exploration rate"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

print("✅ Q-Learning Agent implemented!")
print("\n🎯 Key features:")
print("  - Q-table stores Q(s,a) for all state-action pairs")
print("  - ε-greedy policy balances exploration/exploitation")
print("  - Decaying ε: explore less over time")
print("  - TD learning: update from experience")

In [ ]:
# Train Q-Learning Agent

def train_q_learning(env, agent, num_episodes=500, max_steps=100):
    """
    Train Q-learning agent
    """
    episode_rewards = []
    episode_lengths = []
    epsilon_history = []
    
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        
        for step in range(max_steps):
            # Choose action
            action = agent.get_action(state, training=True)
            
            # Take action
            next_state, reward, done = env.step(action)
            
            # Update Q-table
            agent.update(state, action, reward, next_state, done)
            
            total_reward += reward
            state = next_state
            
            if done:
                break
        
        # Decay epsilon
        agent.decay_epsilon()
        
        # Record metrics
        episode_rewards.append(total_reward)
        episode_lengths.append(step + 1)
        epsilon_history.append(agent.epsilon)
        
        # Print progress
        if (episode + 1) % 100 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            avg_length = np.mean(episode_lengths[-100:])
            print(f"Episode {episode + 1}/{num_episodes} - Avg Reward: {avg_reward:.2f}, Avg Length: {avg_length:.2f}, ε: {agent.epsilon:.3f}")
    
    return episode_rewards, episode_lengths, epsilon_history

# Create agent and environment
env = GridWorld(size=4)
agent = QLearningAgent(
    actions=env.actions,
    learning_rate=0.1,
    discount_factor=0.9,
    epsilon=1.0,
    epsilon_decay=0.995,
    epsilon_min=0.01
)

print("🚀 Training Q-Learning agent...\n")
rewards, lengths, epsilons = train_q_learning(env, agent, num_episodes=500)

print("\n✅ Training complete!")

In [ ]:
# Visualize training progress

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Episode rewards
ax = axes[0, 0]
ax.plot(rewards, alpha=0.3, color='blue', label='Raw')
# Moving average
window = 50
moving_avg = np.convolve(rewards, np.ones(window)/window, mode='valid')
ax.plot(range(window-1, len(rewards)), moving_avg, color='red', linewidth=2, label=f'{window}-episode avg')
ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('Total Reward', fontsize=12)
ax.set_title('📈 Learning Progress: Episode Rewards', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Plot 2: Episode lengths
ax = axes[0, 1]
ax.plot(lengths, alpha=0.3, color='green', label='Raw')
moving_avg_len = np.convolve(lengths, np.ones(window)/window, mode='valid')
ax.plot(range(window-1, len(lengths)), moving_avg_len, color='orange', linewidth=2, label=f'{window}-episode avg')
ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('Steps to Goal', fontsize=12)
ax.set_title('⏱️ Efficiency: Steps per Episode', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Plot 3: Epsilon decay
ax = axes[1, 0]
ax.plot(epsilons, color='purple', linewidth=2)
ax.set_xlabel('Episode', fontsize=12)
ax.set_ylabel('Epsilon (ε)', fontsize=12)
ax.set_title('🔍 Exploration Rate (ε) Decay', fontsize=13, fontweight='bold')
ax.grid(alpha=0.3)
ax.axhline(0.1, color='red', linestyle='--', alpha=0.5, label='10% exploration')
ax.legend()

# Plot 4: Final 50 episodes statistics
ax = axes[1, 1]
final_rewards = rewards[-50:]
ax.hist(final_rewards, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
ax.axvline(np.mean(final_rewards), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(final_rewards):.2f}')
ax.set_xlabel('Total Reward', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('📊 Final 50 Episodes: Reward Distribution', fontsize=13, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Training Summary:")
print(f"  First 50 episodes avg reward: {np.mean(rewards[:50]):.2f}")
print(f"  Last 50 episodes avg reward: {np.mean(rewards[-50:]):.2f}")
print(f"  Improvement: {np.mean(rewards[-50:]) - np.mean(rewards[:50]):.2f}")
print(f"\n  First 50 episodes avg length: {np.mean(lengths[:50]):.1f} steps")
print(f"  Last 50 episodes avg length: {np.mean(lengths[-50:]):.1f} steps")
print(f"\n💡 Agent learned to reach goal more efficiently!")

## 🌟 Real AI Example: Visualizing Learned Policy

**Let's see what the agent learned!**

We'll visualize:
1. **Q-values:** How good each action is in each state
2. **Policy:** Best action in each state (arrows)
3. **Value function:** How valuable each state is

This is exactly how researchers visualize RL agents!

In [ ]:
# Visualize learned policy and Q-values

def visualize_policy(agent, env):
    """
    Visualize learned policy with arrows
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Create value function grid
    value_grid = np.zeros((env.size, env.size))
    
    for i in range(env.size):
        for j in range(env.size):
            state = (i, j)
            if state in env.walls:
                value_grid[i, j] = -10  # Wall marker
            else:
                # State value = max Q-value
                value_grid[i, j] = max(agent.q_table[state].values())
    
    # Plot 1: Value function heatmap
    ax = axes[0]
    im = ax.imshow(value_grid, cmap='RdYlGn', interpolation='nearest')
    
    # Add grid lines
    for i in range(env.size + 1):
        ax.axhline(i - 0.5, color='black', linewidth=2)
        ax.axvline(i - 0.5, color='black', linewidth=2)
    
    # Add values as text
    for i in range(env.size):
        for j in range(env.size):
            if (i, j) in env.walls:
                ax.text(j, i, '🧱', ha='center', va='center', fontsize=25)
            elif (i, j) == env.goal:
                ax.text(j, i, '🎯', ha='center', va='center', fontsize=25)
            else:
                ax.text(j, i, f'{value_grid[i, j]:.1f}', ha='center', va='center', fontsize=11, fontweight='bold')
    
    ax.set_xlim(-0.5, env.size - 0.5)
    ax.set_ylim(env.size - 0.5, -0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('🎯 State Values V(s) = max_a Q(s,a)', fontsize=13, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Value')
    
    # Plot 2: Policy arrows
    ax = axes[1]
    ax.imshow(np.zeros((env.size, env.size)), cmap='gray', vmin=0, vmax=1, alpha=0.1)
    
    # Add grid lines
    for i in range(env.size + 1):
        ax.axhline(i - 0.5, color='black', linewidth=2)
        ax.axvline(i - 0.5, color='black', linewidth=2)
    
    # Arrow directions
    arrow_map = {
        'UP': '↑',
        'DOWN': '↓',
        'LEFT': '←',
        'RIGHT': '→'
    }
    
    # Draw arrows for best action in each state
    for i in range(env.size):
        for j in range(env.size):
            state = (i, j)
            if state in env.walls:
                ax.text(j, i, '🧱', ha='center', va='center', fontsize=25)
            elif state == env.goal:
                ax.text(j, i, '🎯', ha='center', va='center', fontsize=25)
            else:
                # Get best action
                q_values = agent.q_table[state]
                best_action = max(q_values, key=q_values.get)
                arrow = arrow_map[best_action]
                ax.text(j, i, arrow, ha='center', va='center', fontsize=30, fontweight='bold', color='blue')
    
    ax.set_xlim(-0.5, env.size - 0.5)
    ax.set_ylim(env.size - 0.5, -0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('🧭 Learned Policy π(s) = argmax_a Q(s,a)', fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Visualize learned policy
visualize_policy(agent, env)

print("\n🎯 Policy Interpretation:")
print("  Left: State values show 'how good' each state is")
print("  Right: Arrows show best action from each state")
print("  Notice: Arrows point toward goal (optimal path!)")
print("\n💡 This is how DeepMind visualizes AlphaGo's strategy!")

In [ ]:
# Test learned policy (no exploration)

print("🎮 Testing learned policy (exploitation only)...\n")

state = env.reset()
total_reward = 0
path = [state]

print(f"Start: {state}")

for step in range(20):
    # Choose best action (no exploration)
    action = agent.get_action(state, training=False)
    
    # Take action
    next_state, reward, done = env.step(action)
    total_reward += reward
    path.append(next_state)
    
    print(f"Step {step + 1}: {state} --{action:6s}--> {next_state} (reward: {reward:3})")
    
    state = next_state
    
    if done:
        print(f"\n🎉 Reached goal in {step + 1} steps!")
        break

print(f"\nPath taken: {' → '.join([str(s) for s in path])}")
print(f"Total reward: {total_reward}")
print(f"\n💡 Agent learned optimal (or near-optimal) policy!")
print(f"   Optimal path length: {env.size - 1 + env.size - 1} = {2*(env.size-1)} steps (Manhattan distance)")
print(f"   Agent's path: {step + 1} steps")

## 🎯 Interactive Exercises

Test your understanding of RL fundamentals!

### Exercise 1: Modify Reward Structure

**Task:** Change the grid world rewards and retrain

**Experiment with:**
1. Higher goal reward (+100 instead of +10)
2. No step penalty (0 instead of -1)
3. Larger wall penalty (-10 instead of -5)

**Question:** How does each change affect learning?

In [ ]:
# YOUR CODE HERE
# Modify GridWorld class rewards and retrain
# Compare learning curves

pass

<details>
<summary>📖 Click here for solution & insights</summary>

```python
# Higher goal reward:
# - Stronger signal → faster learning
# - But: doesn't change optimal policy

# No step penalty:
# - Agent doesn't care about path length
# - May wander randomly (not efficient)
# - Step penalty encourages shortest path!

# Larger wall penalty:
# - Agent avoids walls more strongly
# - May find safer (but longer) paths
# - Good for robotics (avoid collisions!)
```

**Key Insight:** Reward design (reward shaping) is crucial for RL!
</details>

### Exercise 2: Tune Hyperparameters

**Task:** Experiment with different hyperparameters

**Try:**
1. Learning rate: α = 0.01, 0.1, 0.5, 1.0
2. Discount factor: γ = 0.5, 0.9, 0.99
3. Epsilon decay: 0.99, 0.995, 0.999

**Question:** Which combination learns fastest? Which learns best final policy?

In [ ]:
# YOUR CODE HERE
# Create agents with different hyperparameters
# Train and compare

pass

<details>
<summary>📖 Click here for solution & insights</summary>

```python
# Learning rate (α):
# - Too low (0.01): Slow learning, but stable
# - Too high (1.0): Fast initial learning, but unstable
# - Optimal: 0.1-0.3 for most problems

# Discount factor (γ):
# - Low (0.5): Myopic, only sees nearby rewards
# - High (0.99): Far-sighted, considers long-term
# - For grid world: 0.9 works well

# Epsilon decay:
# - Fast (0.99): Exploits early, may miss better solutions
# - Slow (0.999): Explores longer, finds better policy
# - Trade-off: exploration time vs convergence speed
```

**Best Practice:** Start with standard values (α=0.1, γ=0.9, ε_decay=0.995), then tune!
</details>

### Exercise 3: Calculate Return by Hand

**Given:**
- Reward sequence: [1, 2, 3, 10]
- Discount factor: γ = 0.9

**Task:** Calculate the total discounted return G

**Formula:** G = r₁ + γr₂ + γ²r₃ + γ³r₄

In [ ]:
# YOUR SOLUTION HERE
rewards = [1, 2, 3, 10]
gamma = 0.9

# Calculate G
pass

<details>
<summary>📖 Click here for solution</summary>

```python
G = 1 + 0.9*2 + 0.81*3 + 0.729*10
G = 1 + 1.8 + 2.43 + 7.29
G = 12.52

# Or using code:
G = sum([gamma**i * r for i, r in enumerate(rewards)])
print(f"Return: {G:.2f}")
```

**Notice:** Later rewards count less (7.29 instead of 10)
</details>

## 🎓 Key Takeaways

**You just learned:**

### 1. **What is Reinforcement Learning?**
   - ✅ Learning by trial and error through rewards
   - ✅ Agent-environment interaction loop
   - ✅ Different from supervised/unsupervised learning
   - **Powers:** Game AI, robotics, ChatGPT (RLHF)

### 2. **Markov Decision Processes (MDPs)**
   - ✅ Mathematical framework: (S, A, P, R, γ)
   - ✅ Markov property: future independent of past given present
   - ✅ Discount factor γ balances short/long-term rewards
   - **Used in:** All RL algorithms, planning, control

### 3. **Q-Learning Algorithm**
   - ✅ Learn Q(s,a) = "quality" of actions
   - ✅ Bellman equation: Q*(s,a) = r + γ max Q(s',a')
   - ✅ TD learning: update from experience
   - ✅ ε-greedy: balance exploration/exploitation
   - **Breakthrough:** First RL algorithm to master Atari games (DQN 2013)

### 4. **Key Concepts**
   - ✅ Policy π: strategy for choosing actions
   - ✅ Value function V(s): expected return from state
   - ✅ Exploration vs exploitation trade-off
   - ✅ Reward shaping affects learning

### 🌟 Real-World Impact (2024-2025):

**What You Can Build:**
- 🎮 **Game AI** (Chess, Go, Poker agents)
- 🤖 **Robotics** (Navigation, manipulation)
- 💬 **Chatbots** with RLHF (like ChatGPT)
- 🚗 **Autonomous vehicles** (path planning)
- 📊 **Resource optimization** (energy, traffic)
- 💰 **Trading algorithms** (financial markets)

**Modern Applications:**
- **ChatGPT:** Uses RLHF to align with human preferences
- **AlphaGo:** Beat world champion with self-play RL
- **Tesla Autopilot:** RL for decision making
- **DeepMind:** Energy optimization (40% savings!)
- **OpenAI Five:** Dota 2 world championship
- **Boston Dynamics:** Robots learn to walk/jump

### 📊 Q-Learning vs Other Methods:

| Feature | Q-Learning | Deep Q-Network (DQN) | Policy Gradient |
|---------|------------|---------------------|------------------|
| Function approximation | ❌ Table | ✅ Neural network | ✅ Neural network |
| Discrete actions | ✅ Yes | ✅ Yes | ✅ Yes & continuous |
| Off-policy | ✅ Yes | ✅ Yes | ❌ No (on-policy) |
| Sample efficiency | ✅ Good | ✅ Good | ❌ Poor |
| Stability | ✅ Stable | ⚠️ Can diverge | ⚠️ High variance |
| Used for | Small state spaces | Atari games | Robotics, continuous control |

---

**🎉 Congratulations!** You now understand:
- How RL agents learn from interaction
- The foundations of DeepMind's AlphaGo
- How ChatGPT uses RLHF for alignment
- The core algorithm behind game-playing AI

**Next:** We'll learn Deep Q-Networks (DQN) and modern deep RL! 🚀

## 🚀 Next Steps

**Practice Exercises:**
1. Implement a larger grid world (8x8, 10x10)
2. Add stochastic transitions (actions succeed 80% of time)
3. Implement SARSA (on-policy variant of Q-learning)
4. Try different exploration strategies (Boltzmann, UCB)

**Coming Next:**
- **Day 2:** Deep Q-Networks (DQN), Policy Gradients, Actor-Critic
- **Day 3:** Advanced RL - Multi-agent, AlphaZero, Real-world applications

---

**💡 Deep Dive Resources:**
- Sutton & Barto: "Reinforcement Learning: An Introduction" (THE textbook)
- DeepMind x UCL RL Lecture Series (YouTube)
- OpenAI Spinning Up in Deep RL
- Gymnasium (OpenAI Gym successor) documentation

---

*Remember: RL is how AI learns to master games, control robots, and align language models. You now know the foundations!* 🌟

**🎯 You understand how AlphaGo learned to beat humans at Go!**